In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK for Python: Vertex AI Forecasting Model Training Example


<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_AutoML_Forecasting_Model_Training_Example.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_AutoML_Forecasting_Model_Training_Example.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_AutoML_Forecasting_Model_Training_Example.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This notebook demonstrates how to create an AutoML model based on a time series dataset. It will require you provide a bucket where the dataset will be stored.

Note: You may incur charges for training, prediction, storage or usage of other GCP products in connection with testing this SDK.
### Dataset
This dataset is publicly available in BigQuery. This dataset contains every wholesale purchase of liquor in the State of Iowa by retailers for sale to individuals since January 1, 2012. The training sample contains the sales from 2020 and the prediction sample (used in the batch prediction step) contains the January - April sales from 2021.
Dataset can be found [here](https://console.cloud.google.com/bigquery(cameo:product/iowa-department-of-commerce/iowa-liquor-sales)?project=sue-gcp-learning-env). 

### Objective

In this notebook, you will learn how to create an AutoML model based on a time series dataset.
The steps performed include:

- Create a Managed Time Series Dataset from BigQuery.
- Launch a Training Job to Create a Model.
- Fetch Model Evaluation Metrics.
- Run Batch Prediction.
- Visualize the Forecasts.


### Costs 


This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* BigQuery


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages


In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
%%capture
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
 


### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). {TODO: Update the APIs needed for your tutorial. Edit the API names, and update the link to append the API IDs, separating each one with a comma. For example, container.googleapis.com,cloudbuild.googleapis.com}

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID
Enter your Project ID in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

**If you don't know your project ID**, you may be able to get your project ID using gcloud.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

#### Timestamp


If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket


Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.

You may also change the REGION variable, which is used for operations throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may not use a Multi-Regional Storage bucket for training with Vertex AI.

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

if REGION == "[your-region]":
    REGION = "us-central1"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Import libraries and define constants

In [ ]:
from google.cloud import aiplatform, bigquery

In [ ]:
TRAINING_DATASET_BQ_PATH = (
    "bq://bigquery-public-data:iowa_liquor_sales_forecasting.2020_sales_train"
)

## Initialize Vertex AI SDK

Initialize the *client* for Vertex AI.

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## Create a Vertex Time Series Dataset from a Big Query table

This section will create a dataset from a BigQuery table.

In [ ]:
ds = aiplatform.datasets.TimeSeriesDataset.create(
    display_name="iowa_liquor_sales_train", bq_source=[TRAINING_DATASET_BQ_PATH]
)

ds.resource_name

## Run the training job

Once we have defined your training script, we will create a model.

In [ ]:
time_column = "date"
time_series_identifier_column = "store_name"
target_column = "sale_dollars"

job = aiplatform.AutoMLForecastingTrainingJob(
    display_name="train-iowa-liquor-sales-automl_1",
    optimization_objective="minimize-rmse",
    column_transformations=[
        {"timestamp": {"column_name": time_column}},
        {"numeric": {"column_name": target_column}},
        {"categorical": {"column_name": "city"}},
        {"categorical": {"column_name": "zip_code"}},
        {"categorical": {"column_name": "county"}},
    ],
)

# This will take around an hour to run
model = job.run(
    dataset=ds,
    target_column=target_column,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    available_at_forecast_columns=[time_column],
    unavailable_at_forecast_columns=[target_column],
    time_series_attribute_columns=["city", "zip_code", "county"],
    forecast_horizon=30,
    context_window=30,
    data_granularity_unit="day",
    data_granularity_count=1,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name="iowa-liquor-sales-forecast-model",
    predefined_split_column_name=None,
)

## Get the model evaluation metrics

Fetch the model evaluation metrics calculated during training on the test set.

In [ ]:
import pandas as pd

list_evaluation_pager = model.api_client.list_model_evaluations(
    parent=model.resource_name
)
for model_evaluation in list_evaluation_pager:
    metrics_dict = {m[0]: m[1] for m in model_evaluation.metrics.items()}
    df = pd.DataFrame(metrics_dict.items(), columns=["Metric", "Value"])
    print(df.to_string(index=False))

# Make a batch prediction

### Create a BigQuery dataset for the batch prediction output

First, create a new BigQuery dataset for the batch prediction output in the same region as the batch prediction input dataset. 

In [ ]:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

batch_predict_bq_input_uri = (
    "bq://bigquery-public-data.iowa_liquor_sales_forecasting.2021_sales_predict"
)
batch_predict_bq_output_dataset_name = "iowa_liquor_sales_predictions"
batch_predict_bq_output_dataset_path = "{}.{}".format(
    PROJECT_ID, batch_predict_bq_output_dataset_name
)
batch_predict_bq_output_uri_prefix = "bq://{}.{}".format(
    PROJECT_ID, batch_predict_bq_output_dataset_name
)
# Must be the same region as batch_predict_bq_input_uri
client = bigquery.Client()
dataset = bigquery.Dataset(batch_predict_bq_output_dataset_path)
dataset_region = "US"  # @param {type : "string"}
dataset.location = dataset_region
dataset = client.create_dataset(dataset)
print(
    "Created bigquery dataset {} in {}".format(
        batch_predict_bq_output_dataset_path, dataset_region
    )
)

Run a batch prediction job to generate liquor sales forecasts for stores in Iowa from an input dataset containing historical sales.

In [ ]:
model.batch_predict(
    bigquery_source=batch_predict_bq_input_uri,
    instances_format="bigquery",
    bigquery_destination_prefix=batch_predict_bq_output_uri_prefix,
    predictions_format="bigquery",
    job_display_name="predict-iowa-liquor-sales-automl_1",
)

## Visualize the forecast

Follow the given link to visualize the generated forecasts in [Data Studio](https://support.google.com/datastudio/answer/6283323?hl=en).

In [ ]:
import urllib

tables = client.list_tables(batch_predict_bq_output_dataset_path)

prediction_table_id = ""
for table in tables:
    if (
        table.table_id.startswith("predictions_")
        and table.table_id > prediction_table_id
    ):
        prediction_table_id = table.table_id
batch_predict_bq_output_uri = "{}.{}".format(
    batch_predict_bq_output_dataset_path, prediction_table_id
)


def _sanitize_bq_uri(bq_uri):
    if bq_uri.startswith("bq://"):
        bq_uri = bq_uri[5:]
    return bq_uri.replace(":", ".")


def get_data_studio_link(
    batch_prediction_bq_input_uri,
    batch_prediction_bq_output_uri,
    time_column,
    time_series_identifier_column,
    target_column,
):
    batch_prediction_bq_input_uri = _sanitize_bq_uri(batch_prediction_bq_input_uri)
    batch_prediction_bq_output_uri = _sanitize_bq_uri(batch_prediction_bq_output_uri)
    base_url = "https://datastudio.google.com/c/u/0/reporting"
    query = (
        "SELECT \\n"
        " CAST(input.{} as DATETIME) timestamp_col,\\n"
        " CAST(input.{} as STRING) time_series_identifier_col,\\n"
        " CAST(input.{} as NUMERIC) historical_values,\\n"
        " CAST(predicted_{}.value as NUMERIC) predicted_values,\\n"
        " * \\n"
        "FROM `{}` input\\n"
        "LEFT JOIN `{}` output\\n"
        "ON\\n"
        "CAST(input.{} as DATETIME) = CAST(output.{} as DATETIME)\\n"
        "AND CAST(input.{} as STRING) = CAST(output.{} as STRING)"
    )
    query = query.format(
        time_column,
        time_series_identifier_column,
        target_column,
        target_column,
        batch_prediction_bq_input_uri,
        batch_prediction_bq_output_uri,
        time_column,
        time_column,
        time_series_identifier_column,
        time_series_identifier_column,
    )
    params = {
        "templateId": "067f70d2-8cd6-4a4c-a099-292acd1053e8",
        "ds0.connector": "BIG_QUERY",
        "ds0.projectId": PROJECT_ID,
        "ds0.billingProjectId": PROJECT_ID,
        "ds0.type": "CUSTOM_QUERY",
        "ds0.sql": query,
    }
    params_str_parts = []
    for k, v in params.items():
        params_str_parts.append('"{}":"{}"'.format(k, v))
    params_str = "".join(["{", ",".join(params_str_parts), "}"])
    return "{}?{}".format(base_url, urllib.parse.urlencode({"params": params_str}))


print(
    get_data_studio_link(
        batch_predict_bq_input_uri,
        batch_predict_bq_output_uri,
        time_column,
        time_series_identifier_column,
        target_column,
    )
)

## Clean up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:


In [ ]:
# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

# Delete Vertex AI Dataset
ds.delete()

# Delete the training job
job.delete()

# Delete model resource
model.delete(sync=True)

# Use the delete_contents parameter to delete a dataset and its contents.
# Use the not_found_ok parameter to not receive an error if the dataset has already been deleted.
client.delete_dataset(
    batch_predict_bq_output_dataset_path, delete_contents=True, not_found_ok=True
)  # Make an API request.


if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI